In [2]:
from __future__ import print_function
from __future__ import absolute_import
import cv2
import warnings
import tensorflow as tf
import numpy as np
import random
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Activation, Dense, Input, BatchNormalization, Conv3D, MaxPooling3D, concatenate, AveragePooling3D, Dropout, Reshape ,Lambda, GlobalAveragePooling3D
import os
from keras.utils.data_utils import get_file
from keras import backend as K

In [3]:
# !nvidia-smi

In [4]:
tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)

In [5]:
path = "/home/uas-dtu/Documents/HAR/HMDB51/"
vid_paths = []
labels = []
to_fit = []
for i in os.listdir(path):
    to_fit.append(i)
    for j in os.listdir(path + '/' + i):
        vid = path + '/' + i + '/' + j
        label = i
        vid_paths.append(vid)
        labels.append(label)

In [6]:
# path = "/home/uas-dtu/Documents/HAR/HMDB51"
# vid_paths = []
# labels = []
# to_fit = []
# for i in os.listdir(path):
#     to_fit.append(i)
#     for j in os.listdir(path + '/' + i):
#         video = capture_trim_resize(path + '/' + i + '/' + j, (224, 224), 64, False)
#         label = i
#         vid_paths.append(video.astype(np.float32) / 255)
#         labels.append(label)

In [7]:
print(len(labels), len(vid_paths), len(to_fit))

6779 6779 51


In [8]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit(to_fit)

LabelBinarizer()

In [9]:
# y = lb.transform(label)
# x_train = np.array(vid_paths)
# y_train = np.array(y)

In [10]:
import tensorflow as tf
import numpy as np
import random
import cv2
import os,sys
from PIL import Image
import scipy.misc


def capture_trim_resize(path, shape, n_frames, gray):
  out = []
  cap = cv2.VideoCapture(path)
  valid = True
  for i in range(n_frames):
    _, frame = cap.read()
    if not _:
      frame = np.zeros((shape[0], shape[1], 3), dtype=np.uint8)
    frame = cv2.resize(frame, (shape[1], shape[0]))
    if gray:
      frame = frame[:, :, 0:2]
    out.append(frame)
  return np.array(out)


class DataGenerator(tf.keras.utils.Sequence):
  def __init__(self, x_data, y_data, frame_shape=None, n_frames=None, batch_size=None, shuffle=None):
    self.n_frames = n_frames
    self.shape = frame_shape
    self.batch_size = batch_size
    self.shuffle = shuffle
    self.rgb_videos = x_data
    self.labels = y_data
    self.on_epoch_end()
    
  def on_epoch_end(self):
    if self.shuffle:
      temp = [i for i in range(len(self.labels))]
      random.shuffle(temp)
#       self.flow_videos = [self.flow_videos[i] for i in temp]
      self.rgb_videos = [self.rgb_videos[i] for i in temp]
      self.labels = [self.labels[i] for i in temp]
    

  def __getitem__(self, index):
    start_index = index * self.batch_size
    rgb = []
    y = []
    i = start_index - 1
    while len(y) < self.batch_size:
      try:
        i += 1
        video = capture_trim_resize(self.rgb_videos[i % len(self.rgb_videos)], self.shape, self.n_frames, False)
#         video = np.sum(video)/(255 * 64)
        rgb.append(video.astype(np.float32) / 255)
        y.append(self.labels[i % len(self.rgb_videos)])
      except Exception as err:
        print(err)
        continue
    y = lb.transform(y)
    return np.array(rgb), np.array(y, dtype=np.float32)

  def __len__(self):
    return int(np.floor(len(self.labels) / self.batch_size))

In [11]:
train_data  = DataGenerator(vid_paths, labels, frame_shape=(224, 224), n_frames=64, batch_size=1, shuffle=True)

In [12]:
import random
train_data.__getitem__(random.randint(0, 6779))

(array([[[[[0.00784314, 0.        , 0.00784314],
           [0.02745098, 0.01960784, 0.02352941],
           [0.08235294, 0.07058824, 0.07450981],
           ...,
           [0.15294118, 0.15294118, 0.11764706],
           [0.1254902 , 0.13725491, 0.09803922],
           [0.10588235, 0.13725491, 0.09803922]],
 
          [[0.00784314, 0.        , 0.00392157],
           [0.01960784, 0.00784314, 0.01176471],
           [0.07843138, 0.06666667, 0.07058824],
           ...,
           [0.15294118, 0.15294118, 0.11764706],
           [0.12941177, 0.13725491, 0.10196079],
           [0.10980392, 0.14117648, 0.10196079]],
 
          [[0.        , 0.        , 0.00392157],
           [0.01176471, 0.00392157, 0.00784314],
           [0.0627451 , 0.05490196, 0.05490196],
           ...,
           [0.14901961, 0.15294118, 0.10980392],
           [0.12941177, 0.14117648, 0.09803922],
           [0.11764706, 0.14509805, 0.09803922]],
 
          ...,
 
          [[0.15686275, 0.04313726, 0.431372

In [13]:
path = "/home/uas-dtu/Documents/HAR/validation_hmdb51"
vid_val = []
labels_val = []
to_fit = []
for i in os.listdir(path):
    to_fit.append(i)
    for j in os.listdir(path + '/' + i):
        vid = path + '/' + i + '/' + j
        label = i
        vid_val.append(vid)
        labels_val.append(label)

In [14]:
print(len(vid_val))

511


In [15]:
validation_data = DataGenerator(vid_val, labels_val, frame_shape=(224, 224), n_frames=64, batch_size=1, shuffle=False)

In [16]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [17]:
# def DepthwiseConv3D(input_layer, kernel_size, strides, padding):
#     split = tf.split(input_layer, input_layer.shape[-1], axis = -1)
#     layer = []
#     for i in split:
#         conv = tf.keras.layers.Conv3D(1, kernel_size, strides, padding)(i)
#         layer.append(conv)
#     concat = tf.keras.layers.Concatenate(axis = -1)(layer)
#     conv_f = tf.keras.layers.Conv3D(input_layer.shape[-1], (1, 1, 1))(concat)
#     return conv_f

In [18]:
# def module_1(input_layer, kernel_size, strides, activation, padding = "same"):
#     depth = DepthwiseConv3D(input_layer, kernel_size, strides, padding = padding)
#     bn = tf.keras.layers.BatchNormalization()(depth)
#     activation = tf.keras.layers.Activation(activation)(bn)
#     return activation

# def module_2(input_layer, kernel_size_1, strides_1, kernel_size_2, strides_2, activation_1, activation_2, padding):
#     depth_1 = GroupDepthwiseConv3D(input_layer, kernel_size_1, strides_1)
    
#     bn_1 = tf.keras.layers.BatchNormalization()(depth_1)
#     activation_1 = tf.keras.layers.Activation(activation_1)(bn_1)
#     zero_padding = tf.keras.layers.ZeroPadding3D(padding)(activation_1)
    
#     depth_2 = GroupDepthwiseConv3D(zero_padding, kernel_size_2, strides_2)
    
#     bn_2 = tf.keras.layers.BatchNormalization()(depth_2)
#     activation_2 = tf.keras.layers.Activation(activation_2)(bn_2)
#     return activation_2

# def module_3(input_layer, filters_1, kernel_size_1, strides_1, filters_2, kernel_size_2, strides_2):
#     gap = tf.keras.layers.GlobalAveragePooling3D()(input_layer)
#     rescale = tf.keras.layers.Reshape((1, 1, 1, input_layer.shape[-1]))(gap)
#     conv_1 = tf.keras.layers.Conv3D(filters_1, kernel_size_1, strides=strides_1, padding='valid', groups=1, activation=None)(rescale)
#     conv_2 = tf.keras.layers.Conv3D(filters_2, kernel_size_2, strides=strides_2, padding='valid', groups=1, activation=None)(conv_1)
#     return conv_2

# def stem(input_layer, padding, filters, kernel_size, strides, activations):
#     rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(input_layer)
#     norm = tf.keras.layers.experimental.preprocessing.Normalization(axis=-1)(rescale)
#     zero = tf.keras.layers.ZeroPadding3D(padding)(norm)
#     conv = tf.keras.layers.Conv3D(filters, kernel_size, strides)(zero)
#     bn = tf.keras.layers.BatchNormalization()(conv)
#     activation = tf.keras.layers.Activation(activations)(bn)
#     return activation

# def final(input_layer, filters, kernel_size, strides, padding, activations):
#     conv = tf.keras.layers.Conv3D(filters, kernel_size, strides, padding)(input_layer)
#     bn = tf.keras.layers.BatchNormalization()(conv)
#     activation = tf.keras.layers.Activation(activations)(bn)
#     return activation

In [19]:
# def EfficientNet0_3D(input_layer):
#     start = stem(input_layer, (0, 1, 1), 32, (1, 3, 3), (2, 2, 2), "swish")
#     ### 1a ###
#     mod_1 = module_1(start, (3, 3, 3), (1, 1, 1), "swish", "same")###
#     mod_2 = module_3(mod_1, 8, (1, 1, 1), (1, 1, 1), 32, (1, 1, 1), (1, 1, 1))
#     mul = tf.keras.layers.Multiply()([mod_1, mod_2])
#     conv_1a = tf.keras.layers.Conv3D(16, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
#     bn_1a = tf.keras.layers.BatchNormalization()(conv_1a)
#     ### 2a ###
#     conv_2a = tf.keras.layers.Conv3D(96, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_1a)
#     bn_2a = tf.keras.layers.BatchNormalization()(conv_2a)
#     activation_2a = tf.keras.layers.Activation("swish")(bn_2a)
#     zero_2a = tf.keras.layers.ZeroPadding3D((0, 1, 1))(activation_2a)
#     mod_1 = module_1(zero_2a, (1, 3, 3), (1, 2, 2), "swish", "valid")
# #     mod_1 = tf.keras.layers.Conv3D(mod_1.shape[-1], (1, 1, 1), (2, 1, 1))(mod_1)
#     mod_2 = module_3(mod_1, 4, (1, 1, 1), (1, 1, 1), 96, (1, 1, 1), (1, 1, 1))
#     mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
#     conv_2a2 = tf.keras.layers.Conv3D(24, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
#     bn_2a2 = tf.keras.layers.BatchNormalization()(conv_2a2)
#     ### 2b ###
#     conv_2b = tf.keras.layers.Conv3D(144, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_2a2)
#     bn_2b = tf.keras.layers.BatchNormalization()(conv_2b)
#     activation_2b = tf.keras.layers.Activation("relu")(bn_2b)
#     mod_1 = module_1(activation_2b, (3, 3, 3), (1, 1, 1), "swish")
#     mod_2 = module_3(mod_1, 6, (1, 1, 1), (1, 1, 1), 144, (1, 1, 1), (1, 1, 1))
#     mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
#     conv_2b2 = tf.keras.layers.Conv3D(24, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
#     bn_2b2 = tf.keras.layers.BatchNormalization()(conv_2b2)
#     drop_2b = tf.keras.layers.Dropout(0.2)(bn_2b2)
#     add = tf.keras.layers.Add()([bn_2a2, drop_2b])
    
#     ### 3a ###
#     conv_3a = tf.keras.layers.Conv3D(144, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
#     bn_3a = tf.keras.layers.BatchNormalization()(conv_3a)
#     activation_3a = tf.keras.layers.Activation("swish")(bn_3a)
#     zero_3a = tf.keras.layers.ZeroPadding3D((0, 1, 1))(activation_3a)
#     mod_1 = module_1(zero_3a, (1, 3, 3), (2, 2, 2), "swish", "valid")
# #     mod_1 = tf.keras.layers.Conv3D(mod_1.shape[-1], (1, 1, 1), (2, 1, 1))(mod_1)
#     mod_2 = module_3(mod_1, 6, (1, 1, 1), (1, 1, 1), 144, (1, 1, 1), (1, 1, 1))
#     mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
#     conv_3a2 = tf.keras.layers.Conv3D(40, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
#     bn_3a2 = tf.keras.layers.BatchNormalization()(conv_3a2)
#     ### 3b ###
#     conv_3b = tf.keras.layers.Conv3D(240, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_3a2)
#     bn_3b = tf.keras.layers.BatchNormalization()(conv_3b)
#     activation_3b = tf.keras.layers.Activation("swish")(bn_3b)
#     mod_1 = module_1(activation_3b, (3, 3, 3), (1, 1, 1), "swish")
#     mod_2 = module_3(mod_1, 10, (1, 1, 1), (1, 1, 1), 240, (1, 1, 1), (1, 1, 1))
#     mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
#     conv_3b2 = tf.keras.layers.Conv3D(40, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
#     bn_3b2 = tf.keras.layers.BatchNormalization()(conv_3b2)
#     drop_3b = tf.keras.layers.Dropout(0.2)(bn_3b2)
#     add = tf.keras.layers.Add()([bn_3a2, drop_3b])
#     ### 4a ###
#     conv_4a = tf.keras.layers.Conv3D(240, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
#     bn_4a = tf.keras.layers.BatchNormalization()(conv_4a)
#     activation_4a = tf.keras.layers.Activation("swish")(bn_4a)
#     zero_4a = tf.keras.layers.ZeroPadding3D((0, 1, 1))(activation_4a)
#     mod_1 = module_1(zero_4a, (1, 3, 3), (2, 2, 2), "swish", "valid")
#     mod_2 = module_3(mod_1, 10, (1, 1, 1), (1, 1, 1), 240, (1, 1, 1), (1, 1, 1))#############################
#     mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
#     conv_4a2 = tf.keras.layers.Conv3D(80, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
#     bn_4a2 = tf.keras.layers.BatchNormalization()(conv_4a2)
#     ### 4b ###
#     conv_4b = tf.keras.layers.Conv3D(480, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_4a2)
#     bn_4b = tf.keras.layers.BatchNormalization()(conv_4b)
#     activation_4b = tf.keras.layers.Activation("swish")(bn_4b)
#     mod_1 = module_1(activation_4b, (3, 3, 3), (1, 1, 1), "swish")
#     mod_2 = module_3(mod_1, 20, (1, 1, 1), (1, 1, 1), 480, (1, 1, 1), (1, 1, 1))
#     mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
#     conv_4b2 = tf.keras.layers.Conv3D(80, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
#     bn_4b2 = tf.keras.layers.BatchNormalization()(conv_4b2)
#     drop_4b = tf.keras.layers.Dropout(0.2)(bn_4b2)
#     add = tf.keras.layers.Add()([bn_4a2, drop_4b])
#     ### 4c ###
#     conv_4c = tf.keras.layers.Conv3D(480, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
#     bn_4c = tf.keras.layers.BatchNormalization()(conv_4c)
#     activation_4c = tf.keras.layers.Activation("swish")(bn_4c)
#     mod_1 = module_1(activation_4c, (3, 3, 3), (1, 1, 1), "swish")
#     mod_2 = module_3(mod_1, 20, (1, 1, 1), (1, 1, 1), 480, (1, 1, 1), (1, 1, 1))
#     mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
#     conv_4c2 = tf.keras.layers.Conv3D(80, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
#     bn_4c2 = tf.keras.layers.BatchNormalization()(conv_4c2)
#     drop_4c = tf.keras.layers.Dropout(0.2)(bn_4c2)
#     add = tf.keras.layers.Add()([add, drop_4c])
#     ### 5a ###
#     conv_5a = tf.keras.layers.Conv3D(480, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
#     bn_5a = tf.keras.layers.BatchNormalization()(conv_5a)
#     activation_5a = tf.keras.layers.Activation("swish")(bn_5a)
#     mod_1 = module_1(activation_5a, (3, 3, 3), (1, 1, 1), "swish")
#     mod_2 = module_3(mod_1, 20, (1, 1, 1), (1, 1, 1), 480, (1, 1, 1), (1, 1, 1))
#     mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
#     conv_5a2 = tf.keras.layers.Conv3D(112, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
#     bn_5a2 = tf.keras.layers.BatchNormalization()(conv_5a2)
#     ### 5b ###
#     conv_5b = tf.keras.layers.Conv3D(672, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_5a2)
#     bn_5b = tf.keras.layers.BatchNormalization()(conv_5b)
#     activation_5b = tf.keras.layers.Activation("swish")(bn_5b)
#     mod_1 = module_1(activation_5b, (3, 3, 3), (1, 1, 1), "swish")
#     mod_2 = module_3(mod_1, 28, (1, 1, 1), (1, 1, 1), 672, (1, 1, 1), (1, 1, 1))
#     mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
#     conv_5b2 = tf.keras.layers.Conv3D(112, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
#     bn_5b2 = tf.keras.layers.BatchNormalization()(conv_5b2)
#     drop_5b = tf.keras.layers.Dropout(0.2)(bn_5b2)
#     add = tf.keras.layers.Add()([bn_5a2, drop_5b])
#     ### 5c ###
#     conv_5c = tf.keras.layers.Conv3D(672, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
#     bn_5c = tf.keras.layers.BatchNormalization()(conv_5c)
#     activation_5c = tf.keras.layers.Activation("swish")(bn_5c)
#     mod_1 = module_1(activation_5c, (3, 3, 3), (1, 1, 1), "swish")
#     mod_2 = module_3(mod_1, 28, (1, 1, 1), (1, 1, 1), 672, (1, 1, 1), (1, 1, 1))
#     mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
#     conv_5c2 = tf.keras.layers.Conv3D(112, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
#     bn_5c2 = tf.keras.layers.BatchNormalization()(conv_5c2)
#     drop_5c = tf.keras.layers.Dropout(0.2)(bn_5c2)
#     add = tf.keras.layers.Add()([add, drop_5c])
#     ### 6a ###
#     conv_6a = tf.keras.layers.Conv3D(672, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
#     bn_6a = tf.keras.layers.BatchNormalization()(conv_6a)
#     activation_6a = tf.keras.layers.Activation("swish")(bn_6a)
#     zero_6a = tf.keras.layers.ZeroPadding3D((0, 1, 1))(activation_6a)
#     mod_1 = module_1(zero_6a, (1, 3, 3), (2, 2, 2), "swish", "same")
# #     mod_1 = tf.keras.layers.Conv3D(mod_1.shape[-1], (1, 1, 1), (2, 1, 1))(mod_1)
#     mod_2 = module_3(mod_1, 28, (1, 1, 1), (1, 1, 1), 672, (1, 1, 1), (1, 1, 1))
#     mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
#     conv_6a2 = tf.keras.layers.Conv3D(192, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
#     bn_6a2 = tf.keras.layers.BatchNormalization()(conv_6a2)
#     ### 6b ###
#     conv_6b = tf.keras.layers.Conv3D(1152, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(bn_6a2)
#     bn_6b = tf.keras.layers.BatchNormalization()(conv_6b)
#     activation_6b = tf.keras.layers.Activation("swish")(bn_6b)
#     mod_1 = module_1(activation_6b, (3, 3, 3), (1, 1, 1), "swish")
#     mod_2 = module_3(mod_1, 48, (1, 1, 1), (1, 1, 1), 1152, (1, 1, 1), (1, 1, 1))
#     mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
#     conv_6b2 = tf.keras.layers.Conv3D(192, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
#     bn_6b2 = tf.keras.layers.BatchNormalization()(conv_6b2)
#     drop_6b = tf.keras.layers.Dropout(0.2)(bn_6b2)
#     add = tf.keras.layers.Add()([bn_6a2, drop_6b])
#     ## 6c ###
#     conv_6c = tf.keras.layers.Conv3D(1152, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
#     bn_6c = tf.keras.layers.BatchNormalization()(conv_6c)
#     activation_6c = tf.keras.layers.Activation("swish")(bn_6c)
#     mod_1 = module_1(activation_6c, (3, 3, 3), (1, 1, 1), "swish")
#     mod_2 = module_3(mod_1, 48, (1, 1, 1), (1, 1, 1), 1152, (1, 1, 1), (1, 1, 1))
#     mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
#     conv_6c2 = tf.keras.layers.Conv3D(192, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
#     bn_6c2 = tf.keras.layers.BatchNormalization()(conv_6c2)
#     drop_6c = tf.keras.layers.Dropout(0.2)(bn_6c2)
#     add = tf.keras.layers.Add()([add, drop_6c])
#     ### 6d ###
#     conv_6d = tf.keras.layers.Conv3D(1152, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
#     bn_6d = tf.keras.layers.BatchNormalization()(conv_6d)
#     activation_6d = tf.keras.layers.Activation("swish")(bn_6d)
#     mod_1 = module_1(activation_6d, (3, 3, 3), (1, 1, 1), "swish")
#     mod_2 = module_3(mod_1, 48, (1, 1, 1), (1, 1, 1), 1152, (1, 1, 1), (1, 1, 1))
#     mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
#     conv_6d2 = tf.keras.layers.Conv3D(192, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
#     bn_6d2 = tf.keras.layers.BatchNormalization()(conv_6d2)
#     drop_6d = tf.keras.layers.Dropout(0.2)(bn_6d2)
#     add = tf.keras.layers.Add()([add, drop_6d])
#     ### 7a ###
#     conv_7a = tf.keras.layers.Conv3D(1152, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(add)
#     bn_7a = tf.keras.layers.BatchNormalization()(conv_7a)
#     activation_7a = tf.keras.layers.Activation("swish")(bn_7a)
#     mod_1 = module_1(activation_7a, (3, 3, 3), (1, 1, 1), "swish")
#     mod_2 = module_3(mod_1, 48, (1, 1, 1), (1, 1, 1), 1152, (1, 1, 1), (1, 1, 1))
#     mul = tf.keras.layers.Multiply()([mod_1, mod_2]) 
#     conv_7a2 = tf.keras.layers.Conv3D(320, (1, 1, 1), strides=(1, 1, 1), padding = "valid")(mul)
#     bn_7a2 = tf.keras.layers.BatchNormalization()(conv_7a2)
  
#     end = final(bn_7a2, 1280, (1, 1, 1), (1, 1, 1), "valid", "swish")
    
#     return end

In [20]:
CONV_KERNEL_INITIALIZER = {
    'class_name': 'VarianceScaling',
    'config': {
        'scale': 2.0,
        'mode': 'fan_out',
        'distribution': 'truncated_normal'
    }
}

DENSE_KERNEL_INITIALIZER = {
    'class_name': 'VarianceScaling',
    'config': {
        'scale': 1. / 3.,
        'mode': 'fan_out',
        'distribution': 'uniform'
    }
}

In [21]:
def round_filters(filters, divisor=8):
    filters *= 1
    new_filters = max(divisor, int(filters + divisor / 2) // divisor * divisor)
    if new_filters < 0.9 * filters:
        new_filters += divisor
    return int(new_filters)

def round_repeats(repeats):
    return int(math.ceil(1 * repeats))

In [22]:
# class DepthwiseConv3D(tf.keras.Model):
#   def __init__(self, kernel_size, strides, padding, size):
#     super(DepthwiseConv3D, self).__init__(name='')
#     self.kernel_size = kernel_size
#     self.strides = strides
#     self.padding = padding
#     self.size = size
    
#     self.conv3d = tf.keras.layers.Conv3D(1, self.kernel_size, self.strides, self.padding, kernel_initializer=CONV_KERNEL_INITIALIZER)
#     self.conv_final = tf.keras.layers.Conv3D(self.size, (1, 1, 1), kernel_initializer = CONV_KERNEL_INITIALIZER)
#     self.concat = tf.keras.layers.Concatenate(axis = -1)

#   def call(self, input_tensor, training=False):
#     split = tf.split(input_tensor, input_tensor.shape[-1], axis = -1)
#     layer = []
#     for i in split:
#         conv = self.conv3d(i)
#         layer.append(conv)
#     x = self.concat(layer)
#     x = self.conv_final(x)
    
#     return x

In [23]:
def DepthwiseConv3D(input_layer, kernel_size, strides, padding, size):
    split = tf.split(input_layer, input_layer.shape[-1], axis = -1)
    layer = []
    for i in split:
        conv = tf.keras.layers.Conv3D(1, kernel_size, strides, padding, kernel_initializer=CONV_KERNEL_INITIALIZER)(i)
        layer.append(conv)
    concat = tf.keras.layers.Concatenate(axis = -1)(layer)
    conv_f = tf.keras.layers.Conv3D(input_layer.shape[-1], (1, 1, 1), kernel_initializer=CONV_KERNEL_INITIALIZER)(concat)
    return conv_f

In [24]:
input_layer = tf.keras.layers.Input((64, 224, 224, 3))
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(input_layer)
norm = tf.keras.layers.experimental.preprocessing.Normalization(axis=-1)(rescale)
zero = tf.keras.layers.ZeroPadding3D((0, 1, 1))(norm)
conv = tf.keras.layers.Conv3D(round_filters(32), 3, 2, "valid", kernel_initializer = CONV_KERNEL_INITIALIZER)(zero)
bn = tf.keras.layers.BatchNormalization(axis = 3)(conv)
activation = tf.keras.layers.Activation("swish")(bn)
### 1a ###
x = DepthwiseConv3D(activation, kernel_size = 3, strides = 1, padding = "same", size = 32)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("swish")(x)
se = tf.keras.layers.GlobalAveragePooling3D()(x)
se = tf.keras.layers.Reshape((1, 1, 1, se.shape[-1]))(se)
se = tf.keras.layers.Conv3D(8, 1, padding = "same", activation = "swish")(se)
se = tf.keras.layers.Conv3D(32, 1, padding = "same", activation = "sigmoid")(se)
x = tf.keras.layers.Multiply()([se, x])
x = tf.keras.layers.Conv3D(16, 1,padding='same', use_bias=False, kernel_initializer=CONV_KERNEL_INITIALIZER)(x)
x = tf.keras.layers.BatchNormalization(axis = -1)(x)
### 2a ###
x = tf.keras.layers.Conv3D(96,1,padding='same',use_bias=False,kernel_initializer=CONV_KERNEL_INITIALIZER)(x)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("swish")(x)
x = tf.keras.layers.ZeroPadding3D(padding=(0, 1, 1))(x)
x = DepthwiseConv3D(x, kernel_size = 3, strides = (1, 2, 2), padding = "valid", size = 96)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("swish")(x)
se = tf.keras.layers.GlobalAveragePooling3D()(x)
se = tf.keras.layers.Reshape((1, 1, 1, se.shape[-1]))(se)
se = tf.keras.layers.Conv3D(4,1,padding='same',activation="swish",kernel_initializer=CONV_KERNEL_INITIALIZER)(se)
se = tf.keras.layers.Conv3D(96,1,padding='same',activation='sigmoid',kernel_initializer=CONV_KERNEL_INITIALIZER)(se)
x = tf.keras.layers.Multiply()([x, se])
x = tf.keras.layers.Conv3D(24, 1,padding='same', use_bias=False, kernel_initializer=CONV_KERNEL_INITIALIZER)(x)
x_in = tf.keras.layers.BatchNormalization(axis = -1)(x)
### 2b ###
x = tf.keras.layers.Conv3D(144,1,padding='same',use_bias=False,kernel_initializer=CONV_KERNEL_INITIALIZER)(x_in)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("swish")(x)
x = DepthwiseConv3D(x, kernel_size = 3, strides = 1, padding = "same", size = 144)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("swish")(x)
se = tf.keras.layers.GlobalAveragePooling3D()(x)
se = tf.keras.layers.Reshape((1, 1, 1, se.shape[-1]))(se)
se = tf.keras.layers.Conv3D(6,1,padding='same',activation="swish",kernel_initializer=CONV_KERNEL_INITIALIZER)(se)
se = tf.keras.layers.Conv3D(144,1,padding='same',activation='sigmoid',kernel_initializer=CONV_KERNEL_INITIALIZER)(se)
x = tf.keras.layers.Multiply()([x, se])
x = tf.keras.layers.Conv3D(24, 1,padding='same', use_bias=False, kernel_initializer=CONV_KERNEL_INITIALIZER)(x)
x = tf.keras.layers.BatchNormalization(axis = -1)(x)
x = tf.keras.layers.Dropout(0.1, noise_shape=(None, 1, 1, 1))(x)
x = tf.keras.layers.Add()([x, x_in]) 
### 3a ###
x = tf.keras.layers.Conv3D(144,1,padding='same',use_bias=False,kernel_initializer=CONV_KERNEL_INITIALIZER)(x)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("swish")(x)
x = tf.keras.layers.ZeroPadding3D(padding=(0, 1, 1))(x)
x = DepthwiseConv3D(x, kernel_size = 5, strides = 2, padding = "valid", size = 144)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("swish")(x)
se = tf.keras.layers.GlobalAveragePooling3D()(x)
se = tf.keras.layers.Reshape((1, 1, 1, se.shape[-1]))(se)
se = tf.keras.layers.Conv3D(4,1,padding='same',activation="swish",kernel_initializer=CONV_KERNEL_INITIALIZER)(se)
se = tf.keras.layers.Conv3D(144,1,padding='same',activation='sigmoid',kernel_initializer=CONV_KERNEL_INITIALIZER)(se)
x = tf.keras.layers.Multiply()([x, se])
x = tf.keras.layers.Conv3D(40, 1,padding='same', use_bias=False, kernel_initializer=CONV_KERNEL_INITIALIZER)(x)
x_in = tf.keras.layers.BatchNormalization(axis = -1)(x)
# ### 3b ###
x = tf.keras.layers.Conv3D(240,1,padding='same',use_bias=False,kernel_initializer=CONV_KERNEL_INITIALIZER)(x_in)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("swish")(x)
x = DepthwiseConv3D(x, kernel_size = 5, strides = 1, padding = "same", size = 240)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("swish")(x)
se = tf.keras.layers.GlobalAveragePooling3D()(x)
se = tf.keras.layers.Reshape((1, 1, 1, se.shape[-1]))(se)
se = tf.keras.layers.Conv3D(10,1,padding='same',activation="swish",kernel_initializer=CONV_KERNEL_INITIALIZER)(se)
se = tf.keras.layers.Conv3D(240,1,padding='same',activation='sigmoid',kernel_initializer=CONV_KERNEL_INITIALIZER)(se)
x = tf.keras.layers.Multiply()([x, se])
x = tf.keras.layers.Conv3D(40, 1,padding='same', use_bias=False, kernel_initializer=CONV_KERNEL_INITIALIZER)(x)
x = tf.keras.layers.BatchNormalization(axis = -1)(x)
x = tf.keras.layers.Dropout(0.1, noise_shape=(None, 1, 1, 1))(x)
x = tf.keras.layers.Add()([x, x_in]) 
### 4a ###
x = tf.keras.layers.Conv3D(240,1,padding='same',use_bias=False,kernel_initializer=CONV_KERNEL_INITIALIZER)(x)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("swish")(x)
x = tf.keras.layers.ZeroPadding3D(padding=(0, 1, 1))(x)
x = DepthwiseConv3D(x, kernel_size = 3, strides = 2, padding = "valid", size = 240)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("swish")(x)
se = tf.keras.layers.GlobalAveragePooling3D()(x)
se = tf.keras.layers.Reshape((1, 1, 1, se.shape[-1]))(se)
se = tf.keras.layers.Conv3D(10,1,padding='same',activation="swish",kernel_initializer=CONV_KERNEL_INITIALIZER)(se)
se = tf.keras.layers.Conv3D(240,1,padding='same',activation='sigmoid',kernel_initializer=CONV_KERNEL_INITIALIZER)(se)
x = tf.keras.layers.Multiply()([x, se])
x = tf.keras.layers.Conv3D(80, 1,padding='same', use_bias=False, kernel_initializer=CONV_KERNEL_INITIALIZER)(x)
x_in = tf.keras.layers.BatchNormalization(axis = -1)(x)
### 4b ###
x = tf.keras.layers.Conv3D(480,1,padding='same',use_bias=False,kernel_initializer=CONV_KERNEL_INITIALIZER)(x_in)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("swish")(x)
x = DepthwiseConv3D(x, kernel_size = 3, strides = 1, padding = "same", size = 480)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("swish")(x)
se = tf.keras.layers.GlobalAveragePooling3D()(x)
se = tf.keras.layers.Reshape((1, 1, 1, se.shape[-1]))(se)
se = tf.keras.layers.Conv3D(20,1,padding='same',activation="swish",kernel_initializer=CONV_KERNEL_INITIALIZER)(se)
se = tf.keras.layers.Conv3D(480,1,padding='same',activation='sigmoid',kernel_initializer=CONV_KERNEL_INITIALIZER)(se)
x = tf.keras.layers.Multiply()([x, se])
x = tf.keras.layers.Conv3D(80, 1,padding='same', use_bias=False, kernel_initializer=CONV_KERNEL_INITIALIZER)(x)
x = tf.keras.layers.BatchNormalization(axis = -1)(x)
x = tf.keras.layers.Dropout(0.067, noise_shape=(None, 1, 1, 1))(x)
x_in = tf.keras.layers.Add()([x, x_in]) 
### 4c ###
x = tf.keras.layers.Conv3D(480,1,padding='same',use_bias=False,kernel_initializer=CONV_KERNEL_INITIALIZER)(x_in)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("swish")(x)
x = DepthwiseConv3D(x, kernel_size = 3, strides = 1, padding = "same", size = 480)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("swish")(x)
se = tf.keras.layers.GlobalAveragePooling3D()(x)
se = tf.keras.layers.Reshape((1, 1, 1, se.shape[-1]))(se)
se = tf.keras.layers.Conv3D(20,1,padding='same',activation="swish",kernel_initializer=CONV_KERNEL_INITIALIZER)(se)
se = tf.keras.layers.Conv3D(480,1,padding='same',activation='sigmoid',kernel_initializer=CONV_KERNEL_INITIALIZER)(se)
x = tf.keras.layers.Multiply()([x, se])
x = tf.keras.layers.Conv3D(80, 1,padding='same', use_bias=False, kernel_initializer=CONV_KERNEL_INITIALIZER)(x)
x = tf.keras.layers.BatchNormalization(axis = -1)(x)
x = tf.keras.layers.Dropout(0.13, noise_shape=(None, 1, 1, 1))(x)
x = tf.keras.layers.Add()([x, x_in]) 
### 5a ###
x = tf.keras.layers.Conv3D(480,1,padding='same',use_bias=False,kernel_initializer=CONV_KERNEL_INITIALIZER)(x)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("swish")(x)
x = DepthwiseConv3D(x, kernel_size = 5, strides = 1, padding = "same", size = 480)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("swish")(x)
se = tf.keras.layers.GlobalAveragePooling3D()(x)
se = tf.keras.layers.Reshape((1, 1, 1, se.shape[-1]))(se)
se = tf.keras.layers.Conv3D(20,1,padding='same',activation="swish",kernel_initializer=CONV_KERNEL_INITIALIZER)(se)
se = tf.keras.layers.Conv3D(480,1,padding='same',activation='sigmoid',kernel_initializer=CONV_KERNEL_INITIALIZER)(se)
x = tf.keras.layers.Multiply()([x, se])
x = tf.keras.layers.Conv3D(112, 1,padding='same', use_bias=False, kernel_initializer=CONV_KERNEL_INITIALIZER)(x)
x_in = tf.keras.layers.BatchNormalization(axis = -1)(x)
### 5b ###
x = tf.keras.layers.Conv3D(672,1,padding='same',use_bias=False,kernel_initializer=CONV_KERNEL_INITIALIZER)(x_in)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("swish")(x)
x = DepthwiseConv3D(x, kernel_size = 5, strides = 1, padding = "same", size = 672)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("swish")(x)
se = tf.keras.layers.GlobalAveragePooling3D()(x)
se = tf.keras.layers.Reshape((1, 1, 1, se.shape[-1]))(se)
se = tf.keras.layers.Conv3D(28,1,padding='same',activation="swish",kernel_initializer=CONV_KERNEL_INITIALIZER)(se)
se = tf.keras.layers.Conv3D(672,1,padding='same',activation='sigmoid',kernel_initializer=CONV_KERNEL_INITIALIZER)(se)
x = tf.keras.layers.Multiply()([x, se])
x = tf.keras.layers.Conv3D(112, 1,padding='same', use_bias=False, kernel_initializer=CONV_KERNEL_INITIALIZER)(x)
x = tf.keras.layers.BatchNormalization(axis = -1)(x)
x = tf.keras.layers.Dropout(0.067, noise_shape=(None, 1, 1, 1))(x)
x_in = tf.keras.layers.Add()([x, x_in]) 
### 5c ###
x = tf.keras.layers.Conv3D(672,1,padding='same',use_bias=False,kernel_initializer=CONV_KERNEL_INITIALIZER)(x_in)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("swish")(x)
x = DepthwiseConv3D(x, kernel_size = 5, strides = 1, padding = "same", size = 672)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("swish")(x)
se = tf.keras.layers.GlobalAveragePooling3D()(x)
se = tf.keras.layers.Reshape((1, 1, 1, se.shape[-1]))(se)
se = tf.keras.layers.Conv3D(28,1,padding='same',activation="swish",kernel_initializer=CONV_KERNEL_INITIALIZER)(se)
se = tf.keras.layers.Conv3D(672,1,padding='same',activation='sigmoid',kernel_initializer=CONV_KERNEL_INITIALIZER)(se)
x = tf.keras.layers.Multiply()([x, se])
x = tf.keras.layers.Conv3D(112, 1,padding='same', use_bias=False, kernel_initializer=CONV_KERNEL_INITIALIZER)(x)
x = tf.keras.layers.BatchNormalization(axis = -1)(x)
x = tf.keras.layers.Dropout(0.067, noise_shape=(None, 1, 1, 1))(x)
x = tf.keras.layers.Add()([x, x_in])
### 6a ###
x = tf.keras.layers.Conv3D(672,1,padding='same',use_bias=False,kernel_initializer=CONV_KERNEL_INITIALIZER)(x)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("swish")(x)
x = tf.keras.layers.ZeroPadding3D(padding=(0, 2, 2))(x)
x = DepthwiseConv3D(x, kernel_size = 5, strides = 2, padding = "valid", size = 672)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("swish")(x)
se = tf.keras.layers.GlobalAveragePooling3D()(x)
se = tf.keras.layers.Reshape((1, 1, 1, se.shape[-1]))(se)
se = tf.keras.layers.Conv3D(28,1,padding='same',activation="swish",kernel_initializer=CONV_KERNEL_INITIALIZER)(se)
se = tf.keras.layers.Conv3D(672,1,padding='same',activation='sigmoid',kernel_initializer=CONV_KERNEL_INITIALIZER)(se)
x = tf.keras.layers.Multiply()([x, se])
x = tf.keras.layers.Conv3D(192, 1,padding='same', use_bias=False, kernel_initializer=CONV_KERNEL_INITIALIZER)(x)
x_in = tf.keras.layers.BatchNormalization(axis = -1)(x)
### 6b ###
x = tf.keras.layers.Conv3D(1152,1,padding='same',use_bias=False,kernel_initializer=CONV_KERNEL_INITIALIZER)(x_in)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("swish")(x)
x = DepthwiseConv3D(x, kernel_size = 5, strides = 1, padding = "same", size = 1152)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("swish")(x)
se = tf.keras.layers.GlobalAveragePooling3D()(x)
se = tf.keras.layers.Reshape((1, 1, 1, se.shape[-1]))(se)
se = tf.keras.layers.Conv3D(48,1,padding='same',activation="swish",kernel_initializer=CONV_KERNEL_INITIALIZER)(se)
se = tf.keras.layers.Conv3D(1152,1,padding='same',activation='sigmoid',kernel_initializer=CONV_KERNEL_INITIALIZER)(se)
x = tf.keras.layers.Multiply()([x, se])
x = tf.keras.layers.Conv3D(192, 1,padding='same', use_bias=False, kernel_initializer=CONV_KERNEL_INITIALIZER)(x)
x = tf.keras.layers.BatchNormalization(axis = -1)(x)
x = tf.keras.layers.Dropout(0.05, noise_shape=(None, 1, 1, 1))(x)
x_in = tf.keras.layers.Add()([x, x_in]) 
### 6c ###
x = tf.keras.layers.Conv3D(1152,1,padding='same',use_bias=False,kernel_initializer=CONV_KERNEL_INITIALIZER)(x_in)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("swish")(x)
x = DepthwiseConv3D(x, kernel_size = 5, strides = 1, padding = "same", size = 1152)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("swish")(x)
se = tf.keras.layers.GlobalAveragePooling3D()(x)
se = tf.keras.layers.Reshape((1, 1, 1, se.shape[-1]))(se)
se = tf.keras.layers.Conv3D(48,1,padding='same',activation="swish",kernel_initializer=CONV_KERNEL_INITIALIZER)(se)
se = tf.keras.layers.Conv3D(1152,1,padding='same',activation='sigmoid',kernel_initializer=CONV_KERNEL_INITIALIZER)(se)
x = tf.keras.layers.Multiply()([x, se])
x = tf.keras.layers.Conv3D(192, 1,padding='same', use_bias=False, kernel_initializer=CONV_KERNEL_INITIALIZER)(x)
x = tf.keras.layers.BatchNormalization(axis = -1)(x)
x = tf.keras.layers.Dropout(0.1, noise_shape=(None, 1, 1, 1))(x)
x_in = tf.keras.layers.Add()([x, x_in]) 
### 6d ###
x = tf.keras.layers.Conv3D(1152,1,padding='same',use_bias=False,kernel_initializer=CONV_KERNEL_INITIALIZER)(x_in)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("swish")(x)
x = DepthwiseConv3D(x, kernel_size = 5, strides = 1, padding = "same", size = 1152)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("swish")(x)
se = tf.keras.layers.GlobalAveragePooling3D()(x)
se = tf.keras.layers.Reshape((1, 1, 1, se.shape[-1]))(se)
se = tf.keras.layers.Conv3D(48,1,padding='same',activation="swish",kernel_initializer=CONV_KERNEL_INITIALIZER)(se)
se = tf.keras.layers.Conv3D(1152,1,padding='same',activation='sigmoid',kernel_initializer=CONV_KERNEL_INITIALIZER)(se)
x = tf.keras.layers.Multiply()([x, se])
x = tf.keras.layers.Conv3D(192, 1,padding='same', use_bias=False, kernel_initializer=CONV_KERNEL_INITIALIZER)(x)
x = tf.keras.layers.BatchNormalization(axis = -1)(x)
x = tf.keras.layers.Dropout(0.15, noise_shape=(None, 1, 1, 1))(x)
x = tf.keras.layers.Add()([x, x_in]) 
### 7a ###
x = tf.keras.layers.Conv3D(1152,1,padding='same',use_bias=False,kernel_initializer=CONV_KERNEL_INITIALIZER)(x)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("swish")(x)
x = DepthwiseConv3D(x, kernel_size = 3, strides = 1, padding = "same", size = 1152)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("swish")(x)
se = tf.keras.layers.GlobalAveragePooling3D()(x)
se = tf.keras.layers.Reshape((1, 1, 1, se.shape[-1]))(se)
se = tf.keras.layers.Conv3D(48,1,padding='same',activation="swish",kernel_initializer=CONV_KERNEL_INITIALIZER)(se)
se = tf.keras.layers.Conv3D(1152,1,padding='same',activation='sigmoid',kernel_initializer=CONV_KERNEL_INITIALIZER)(se)
x = tf.keras.layers.Multiply()([x, se])
x = tf.keras.layers.Conv3D(320, 1,padding='same', use_bias=False, kernel_initializer=CONV_KERNEL_INITIALIZER)(x)
x_in = tf.keras.layers.BatchNormalization(axis = -1)(x)
### TOP ###
x = tf.keras.layers.Conv3D(round_filters(1280),1,padding='same',use_bias=False,kernel_initializer=CONV_KERNEL_INITIALIZER)(x)
x = tf.keras.layers.BatchNormalization(axis=-1)(x)
x = tf.keras.layers.Activation("swish")(x)
x = tf.keras.layers.GlobalAveragePooling3D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(51,activation="softmax",kernel_initializer=DENSE_KERNEL_INITIALIZER)(x)

In [25]:
model = tf.keras.models.Model(inputs = input_layer, outputs = x)

In [26]:
# input_rgb = tf.keras.layers.Input((64, 224, 224, 3))
# eff_rgb = EfficientNet0_3D(input_rgb)
# print(eff_rgb.shape)

In [27]:
# avg = tf.keras.layers.AveragePooling3D((1, 8, 8), strides = (1, 1, 1), padding = "valid")(eff_rgb)
# conv = tf.keras.layers.Conv3D(51, (1, 1, 1))(avg)
# print(conv.shape)

In [28]:
# rgb_avg = tf.keras.layers.AveragePooling3D((1, 8, 8), strides = (1, 1, 1), padding = "valid")(eff_rgb)
# rgb_conv = tf.keras.layers.Conv3D(51, (1, 1, 1))(rgb_avg)
# flatten = tf.keras.layers.Flatten()(rgb_conv)
# dense = tf.keras.layers.Dense(51)(flatten)
# activation = tf.keras.layers.Activation("softmax")(dense)

In [29]:
# model = tf.keras.models.Model(inputs = input_rgb, outputs = activation)

In [30]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 224, 224 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 64, 224, 224, 0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 64, 224, 224, 7           rescaling[0][0]                  
__________________________________________________________________________________________________
zero_padding3d (ZeroPadding3D)  (None, 64, 226, 226, 0           normalization[0][0]              
______________________________________________________________________________________________

conv3d_296 (Conv3D)             (None, 13, 27, 27, 1 126         tf.split_3[0][8]                 
__________________________________________________________________________________________________
conv3d_297 (Conv3D)             (None, 13, 27, 27, 1 126         tf.split_3[0][9]                 
__________________________________________________________________________________________________
conv3d_298 (Conv3D)             (None, 13, 27, 27, 1 126         tf.split_3[0][10]                
__________________________________________________________________________________________________
conv3d_299 (Conv3D)             (None, 13, 27, 27, 1 126         tf.split_3[0][11]                
__________________________________________________________________________________________________
conv3d_300 (Conv3D)             (None, 13, 27, 27, 1 126         tf.split_3[0][12]                
__________________________________________________________________________________________________
conv3d_301

conv3d_681 (Conv3D)             (None, 13, 27, 27, 2 9600        add_1[0][0]                      
__________________________________________________________________________________________________
batch_normalization_15 (BatchNo (None, 13, 27, 27, 2 960         conv3d_681[0][0]                 
__________________________________________________________________________________________________
activation_10 (Activation)      (None, 13, 27, 27, 2 0           batch_normalization_15[0][0]     
__________________________________________________________________________________________________
zero_padding3d_3 (ZeroPadding3D (None, 13, 29, 29, 2 0           activation_10[0][0]              
__________________________________________________________________________________________________
tf.split_5 (TFOpLambda)         [(None, 13, 29, 29,  0           zero_padding3d_3[0][0]           
__________________________________________________________________________________________________
conv3d_682

conv3d_1129 (Conv3D)            (None, 6, 14, 14, 1) 28          tf.split_6[0][202]               
__________________________________________________________________________________________________
conv3d_1130 (Conv3D)            (None, 6, 14, 14, 1) 28          tf.split_6[0][203]               
__________________________________________________________________________________________________
conv3d_1131 (Conv3D)            (None, 6, 14, 14, 1) 28          tf.split_6[0][204]               
__________________________________________________________________________________________________
conv3d_1132 (Conv3D)            (None, 6, 14, 14, 1) 28          tf.split_6[0][205]               
__________________________________________________________________________________________________
conv3d_1133 (Conv3D)            (None, 6, 14, 14, 1) 28          tf.split_6[0][206]               
__________________________________________________________________________________________________
conv3d_113

conv3d_1593 (Conv3D)            (None, 6, 14, 14, 1) 28          tf.split_7[0][181]               
__________________________________________________________________________________________________
conv3d_1594 (Conv3D)            (None, 6, 14, 14, 1) 28          tf.split_7[0][182]               
__________________________________________________________________________________________________
conv3d_1595 (Conv3D)            (None, 6, 14, 14, 1) 28          tf.split_7[0][183]               
__________________________________________________________________________________________________
conv3d_1596 (Conv3D)            (None, 6, 14, 14, 1) 28          tf.split_7[0][184]               
__________________________________________________________________________________________________
conv3d_1597 (Conv3D)            (None, 6, 14, 14, 1) 28          tf.split_7[0][185]               
__________________________________________________________________________________________________
conv3d_159

...

                                                                 conv3d_6915[0][0]                
                                                                 conv3d_6916[0][0]                
                                                                 conv3d_6917[0][0]                
                                                                 conv3d_6918[0][0]                
                                                                 conv3d_6919[0][0]                
                                                                 conv3d_6920[0][0]                
                                                                 conv3d_6921[0][0]                
                                                                 conv3d_6922[0][0]                
                                                                 conv3d_6923[0][0]                
                                                                 conv3d_6924[0][0]                
          

conv3d_8182 (Conv3D)            (None, 1, 7, 7, 1)   28          tf.split_15[0][298]              
__________________________________________________________________________________________________
conv3d_8183 (Conv3D)            (None, 1, 7, 7, 1)   28          tf.split_15[0][299]              
__________________________________________________________________________________________________
conv3d_8184 (Conv3D)            (None, 1, 7, 7, 1)   28          tf.split_15[0][300]              
__________________________________________________________________________________________________
conv3d_8185 (Conv3D)            (None, 1, 7, 7, 1)   28          tf.split_15[0][301]              
__________________________________________________________________________________________________
conv3d_8186 (Conv3D)            (None, 1, 7, 7, 1)   28          tf.split_15[0][302]              
__________________________________________________________________________________________________
conv3d_818

conv3d_8583 (Conv3D)            (None, 1, 7, 7, 1)   28          tf.split_15[0][699]              
__________________________________________________________________________________________________
conv3d_8584 (Conv3D)            (None, 1, 7, 7, 1)   28          tf.split_15[0][700]              
__________________________________________________________________________________________________
conv3d_8585 (Conv3D)            (None, 1, 7, 7, 1)   28          tf.split_15[0][701]              
__________________________________________________________________________________________________
conv3d_8586 (Conv3D)            (None, 1, 7, 7, 1)   28          tf.split_15[0][702]              
__________________________________________________________________________________________________
conv3d_8587 (Conv3D)            (None, 1, 7, 7, 1)   28          tf.split_15[0][703]              
__________________________________________________________________________________________________
conv3d_858

                                                                 conv3d_7929[0][0]                
                                                                 conv3d_7930[0][0]                
                                                                 conv3d_7931[0][0]                
                                                                 conv3d_7932[0][0]                
                                                                 conv3d_7933[0][0]                
                                                                 conv3d_7934[0][0]                
                                                                 conv3d_7935[0][0]                
                                                                 conv3d_7936[0][0]                
                                                                 conv3d_7937[0][0]                
                                                                 conv3d_7938[0][0]                
          

Total params: 12,335,156
Trainable params: 12,293,613
Non-trainable params: 41,543
__________________________________________________________________________________________________


In [30]:
tf.keras.utils.plot_model(model)

In [31]:
# def scheduler(epoch, lr):
#   print("previous learning rate:",lr)
#   initial = 0.001
#   for i in range(epoch):
#     initial = initial * 0.95
#   return initial

from tensorflow.keras.optimizers import Adam
optimizer=Adam(0.0001)

In [32]:
model.compile(optimizer=optimizer, loss= 'categorical_crossentropy', metrics=['accuracy'])

In [33]:
from tensorflow.keras.callbacks import (
    ModelCheckpoint,
    TensorBoard
)
import tensorflow as tf

In [34]:
callbacks = [tf.keras.callbacks.ModelCheckpoint("/home/uas-dtu/Documents/HAR/test_weights/train_{epoch}.tf", verbose = 1,
                                                save_weights_only=True),
             TensorBoard("/home/uas-dtu/Documents/HAR/logs/combined_logs")]

In [35]:
# callbacks = [tf.keras.callbacks.ModelCheckpoint("/home/uas-dtu/Documents/HAR/checkpoints_5classes/train_{epoch}.tf", verbose = 1,save_weights_only=True)]#, tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=1)]

In [36]:
!nvidia-smi

Tue Jan  4 19:58:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P5000        Off  | 00000000:73:00.0  On |                  Off |
| 26%   39C    P0    43W / 180W |  15802MiB / 16270MiB |      8%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
model.load_weights("/home/uas-dtu/Documents/HAR/test_weights/train_48.tf")

In [ ]:
model.fit(train_data, epochs = 70, verbose=1,callbacks = callbacks, initial_epoch=48)

/home/uas-dtu/.local/lib/python3.6/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 49/70
6779/6779 [==============================] - 17136s 2s/step - loss: 0.9816 - accuracy: 0.7200 - val_loss: 28.3861 - val_accuracy: 0.0254

Epoch 00049: saving model to /home/uas-dtu/Documents/HAR/test_weights/train_49.tf
Epoch 50/70
6779/6779 [==============================] - 16619s 2s/step - loss: 0.8842 - accuracy: 0.7426 - val_loss: 31.3955 - val_accuracy: 0.0235

Epoch 00050: saving model to /home/uas-dtu/Documents/HAR/test_weights/train_50.tf
Epoch 51/70
6779/6779 [==============================] - 16659s 2s/step - loss: 0.7742 - accuracy: 0.7784 - val_loss: 33.5455 - val_accuracy: 0.0117

Epoch 00051: saving model to /home/uas-dtu/Documents/HAR/test_weights/train_51.tf
Epoch 52/70
6779/6779 [==============================] - 16587s 2s/step - loss: 0.6683 - accuracy: 0.8132 - val_loss: 58.2154 - val_accuracy: 0.0157

Epoch 00052: saving model to /home/uas-dtu/Documents/HAR/test_weights/train_52.tf
Epoch 53/70
6779/6779 [==============================] - 16418s 2s/step 

In [ ]:
model.load_weights("/home/uas-dtu/Documents/HAR/test_weights/train_43.tf")

In [ ]:
pred = model.predict(train_data.__getitem__(1)[0])

In [ ]:
print(pred)
print(np.argmax(pred))
print(np.argmax(pred))

In [ ]:
print(np.argmax(pred))